In [4]:
import tkinter as tk
from tkinter import messagebox

# 상수 정의
BOARD_SIZE = 15
CELL_SIZE = 40
LINE_WIDTH = 2
WINDOW_WIDTH = BOARD_SIZE * CELL_SIZE + LINE_WIDTH
WINDOW_HEIGHT = BOARD_SIZE * CELL_SIZE + LINE_WIDTH

# 오목판 초기화
board = [['.' for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

# Tkinter 애플리케이션 및 윈도우 생성
root = tk.Tk()
root.title("오목 게임")

# 캔버스 생성
canvas = tk.Canvas(root, width=WINDOW_WIDTH, height=WINDOW_HEIGHT, bg='white')
canvas.pack()

# 오목판 그리기
def draw_board():
    for i in range(BOARD_SIZE):
        canvas.create_line(0, i * CELL_SIZE + LINE_WIDTH / 2, WINDOW_WIDTH - LINE_WIDTH, i * CELL_SIZE + LINE_WIDTH / 2, width=LINE_WIDTH)
        canvas.create_line(i * CELL_SIZE + LINE_WIDTH / 2, 0, i * CELL_SIZE + LINE_WIDTH / 2, WINDOW_HEIGHT - LINE_WIDTH, width=LINE_WIDTH)

# 돌 그리기
def draw_stone(row, col, player):
    x = col * CELL_SIZE + LINE_WIDTH / 2
    y = row * CELL_SIZE + LINE_WIDTH / 2
    if player == 'X':
        canvas.create_oval(x - CELL_SIZE // 3, y - CELL_SIZE // 3, x + CELL_SIZE // 3, y + CELL_SIZE // 3, fill='black')
    else:
        canvas.create_oval(x - CELL_SIZE // 3, y - CELL_SIZE // 3, x + CELL_SIZE // 3, y + CELL_SIZE // 3, fill='white')

# 클릭 이벤트 핸들러
def click(event):
    col = round(event.x / CELL_SIZE)
    row = round(event.y / CELL_SIZE)
    player = 'X' if len(canvas.find_all()) % 2 == 0 else 'O'  # 번갈아가며 플레이어 교체
    if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and board[row][col] == '.':
        if player == 'X':
            if check_renju_rule(row, col):
                messagebox.showwarning("렌주룰 위반", "렌주룰에 위반됩니다. 다른 곳에 돌을 놓아주세요.")
                return
        board[row][col] = player
        draw_stone(row, col, player)
        if check_win(row, col, player):
            messagebox.showinfo("게임 종료", f"{player} 플레이어가 승리했습니다!")
            reset_game()
            return

# 렌주룰 체크
def check_renju_rule(row, col):
    # 흑돌인 경우에만 쌍삼, 사사 체크
    if check_sam_sam(row, col, 'X'):
        return True
    return False

# 삼삼 체크
def check_sam_sam(row, col, player):
    directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
    for dr, dc in directions:
        count = 1
        for step in range(1, 5):
            r = row + dr * step
            c = col + dc * step
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r][c] == player:
                count += 1
            else:
                break
        for step in range(1, 5):
            r = row - dr * step
            c = col - dc * step
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r][c] == player:
                count += 1
            else:
                break
        if count >= 3:
            return True
    return False

# 승리 조건 확인
def check_win(row, col, player):
    # 가로, 세로, 대각선 방향으로 연속된 돌이 5개인지 확인
    directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
    for dr, dc in directions:
        count = 1
        for step in range(1, 5):
            r = row + dr * step
            c = col + dc * step
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r][c] == player:
                count += 1
            else:
                break
        for step in range(1, 5):
            r = row - dr * step
            c = col - dc * step
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and board[r][c] == player:
                count += 1
            else:
                break
        if count >= 5:
            return True
    return False

# 게임 리셋
def reset_game():
    global board
    board = [['.' for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    canvas.delete("all")
    draw_board()

# 오목판 그리기
draw_board()

# 캔버스 클릭 이벤트 바인딩
canvas.bind("<Button-1>", click)

# 'c' 키를 눌러서 강제 종료하는 함수
def force_quit(event):
    if event.char == 'c':
        root.destroy()

# 키 이벤트 바인딩
root.bind("<Key>", force_quit)

# 애플리케이션 실행
root.mainloop()